In [107]:
"""
Disclaimer: This file purely exists to parse the fam_masterlist.csv file. It does not actively run during production.
Use instructions: Run all cells in this file to generate a fam_masterlist.json file. This file will be used in the main program.

For all questions, contact Sam at @smmzhu or smmzhu@gmail.com or samuelzhu@ucsb.org
""" 


'\nDisclaimer: This file purely exists to parse the fam_masterlist.csv file. It does not actively run during production.\nUse instructions: Run all cells in this file to generate a fam_masterlist.json file. This file will be used in the main program.\n\nFor all questions, contact Sam at @smmzhu or smmzhu@gmail.com or samuelzhu@ucsb.org\n'

In [108]:
"""
Since python does not have fields, we can just define the fields that dicts will have, and then encode them into json

Fam object:
- heads: Person[], where each person is the root node of a family tree (theoretically should be one, maybe two/three for edge cases)
- fam_name: string

Person object:
- big: string
- littles: string[]
- recruitment_year: int
- fam_name: string

Return json (list of Fam objects):
[{
    "Fam": string,
    "Heads": string[],
    "Members": [
        {
            "big": string,
            "littles": string[],
            "recruitment_year": int,
            "fam_name": string
        },
        ...
    ]
}, ...]
"""

'\nSince python does not have fields, we can just define the fields that dicts will have, and then encode them into json\n\nFam object:\n- heads: Person[], where each person is the root node of a family tree (theoretically should be one, maybe two/three for edge cases)\n- fam_name: string\n\nPerson object:\n- big: string\n- littles: string[]\n- recruitment_year: int\n- fam_name: string\n\nReturn json (list of Fam objects):\n[{\n    "Fam": string,\n    "Heads": string[],\n    "Members": [\n        {\n            "big": string,\n            "littles": string[],\n            "recruitment_year": int,\n            "fam_name": string\n        },\n        ...\n    ]\n}, ...]\n'

In [109]:
import pandas as pd
df = pd.read_csv('fam_masterlist.csv')
df = df[["Name", "Recruitment Year", "Big", "Fam"]]
df.drop(df[df["Fam"] == "Unknown"].index, inplace=True)
df.drop(df[df["Fam"] == "n/a"].index, inplace=True)
df.drop(df[df["Fam"] == "none"].index, inplace=True)
df.drop(df[df["Fam"].isna()].index, inplace=True)

#make all fields lowercase, and trim whitespace
df["Big"] = df["Big"].str.lower().str.strip()
df["Fam"] = df["Fam"].str.lower().str.strip()
df["Name"] = df["Name"].str.lower().str.strip()

In [110]:
#add a preprocessing step to add people with no bigs as entries:
for big, fam in df[["Big", "Fam"]].values:
    print(big, fam)
    if big not in df[df["Fam"] == fam]["Name"].values:
        df.loc[-1] = [big, "None", "None", fam]
        df.index = df.index + 1
        df = df.sort_index()

jason xie hufuwu
jason xie hufuwu
casidy chen hufuwu
jessica wang hufuwu
jason xie hufuwu
jason xie hufuwu
yoobin won hufuwu
yoobin won hufuwu
andrew cheng hufuwu
andrew cheng hufuwu
katie kao hufuwu
jackie chen hufuwu
alison chen hufuwu
jessica wang hufuwu
jessica wang hufuwu
wynee chen hufuwu
jason xie hufuwu
andrew cheng hufuwu
andrew cheng hufuwu
andrew cheng hufuwu
lily li hufuwu
lily li hufuwu
jackie chen hufuwu
jackie chen hufuwu
jackie chen hufuwu
johnson chan hufuwu
breeze tucker hufuwu
breeze tucker hufuwu
claire lee hufuwu
claire lee hufuwu
ian huang hufuwu
ian huang hufuwu
ian huang hufuwu
harold mo hufuwu
harold mo hufuwu
jasper chen hufuwu
jasper chen hufuwu
andrew cheng hufuwu
aaron lee hufuwu
amy wang hufuwu
amy wang hufuwu
anna chuang hufuwu
ben chen hufuwu
ben chen hufuwu
cappillen lee hufuwu
cappillen lee hufuwu
cappillen lee hufuwu
chauncey vuong hufuwu
david nhan hufuwu
david nhan hufuwu
eshan joshi hufuwu
eshan joshi hufuwu
fiona hosmer-hughes hufuwu
fiona hosmer-

In [111]:
#add a preprocessing step — if the year is empty, set it to "None"
df["Recruitment Year"].fillna("None", inplace=True)

In [112]:
df

Name Recruitment Year          Big     Fam
0    nicholas jan             None         None  lu rou
1     ee-ning ooi             None         None  lu rou
2      andrea nei             None         None  lu rou
3    johnson chan             None         None   texas
4       aaron tai             None         None   texas
..            ...              ...          ...     ...
572  daiji uchino             2024  samuel zhou  lu rou
573    vivian bui              W25   kevin wang  lu rou
574     nyel azam              W25   kevin wang  lu rou
575    amanda lim              W25    alan sheu  lu rou
576     ian chung              W25    alan sheu  lu rou

[571 rows x 4 columns]

In [113]:
#create a dictionary of families, key is the family name, value is a list of all the members
fam_people = {}
for index, row in df.iterrows():
    if row["Fam"] not in fam_people:
        fam_people[row["Fam"]] = []
    fam_people[row["Fam"]].append(
        {
            "Name": row["Name"],
            "Recruitment Year": row["Recruitment Year"],
            "Big": row["Big"],
            "Fam": row["Fam"]
        }
    )

fam_name_list = list(fam_people.keys())

final_fam_return = []

#per fam:
for fam_name in fam_name_list:
    fam = fam_people[fam_name]

    #well connect them (you just have to populate the littles field)
    for person in fam:
        person["Little"] = []
        for person2 in fam:
            if person["Name"] == person2["Big"]:
                person["Little"].append(person2["Name"])
    
    #find the head(s) of the fam
    heads = []
    fam_names = [x["Name"] for x in fam]
    for person in fam:
        top_name = person["Name"]
        for x in range(100000):
            if top_name in fam_names:
                temp = [x for x in fam_people[fam_name] if x["Name"] == top_name][0]["Big"]
                if temp != "None":
                    top_name = temp
                else:
                    break
            else:
                break
            
        print("found top name: ", top_name, "in the family: ", fam_name)
        if not top_name in heads: 
            heads.append(top_name)

    #store them in the master list
    final_fam_return.append(
        {
            "Fam": fam_name,
            "Heads": heads.copy(),
            "Members": fam
        }
    )




found top name:  nicholas jan in the family:  lu rou
found top name:  ee-ning ooi in the family:  lu rou
found top name:  andrea nei in the family:  lu rou
found top name:  andrea nei in the family:  lu rou
found top name:  andrea nei in the family:  lu rou
found top name:  ee-ning ooi in the family:  lu rou
found top name:  ee-ning ooi in the family:  lu rou
found top name:  ee-ning ooi in the family:  lu rou
found top name:  nicholas jan in the family:  lu rou
found top name:  nicholas jan in the family:  lu rou
found top name:  ee-ning ooi in the family:  lu rou
found top name:  ee-ning ooi in the family:  lu rou
found top name:  ee-ning ooi in the family:  lu rou
found top name:  ee-ning ooi in the family:  lu rou
found top name:  ee-ning ooi in the family:  lu rou
found top name:  johnson chan in the family:  texas
found top name:  aaron tai in the family:  texas
found top name:  aaron tai in the family:  texas
found top name:  aaron tai in the family:  texas
found top name:  aaro

In [114]:
for item in final_fam_return:
    if item["Fam"] == "best fam":
        print(item["Heads"])
    # print(len(item["Heads"]))

final_fam_return.sort(key=lambda x: len(x["Members"]), reverse=True)

['andrew xiong', 'shawn li', 'andrea wang', 'ashley gong', 'lauren leung']


In [115]:
import json
f = open('data.json', 'wb')
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(final_fam_return, f, ensure_ascii=False, indent=4)